# Programmatically Create Dashboard using Arize's GrapQL API

This tutorial demonstrates how to create a custom dashboard using Arize's GraphQL API. Here's what we will cover:

- Fetching model metadata and features.
- Creating different types of widgets such as bar charts, line charts, and drift widgets.
- Placing these widgets on a dashboard in a specified order.

**Important Notes:**
- Although line charts and distributions can have multiple plots, in this example, we will add only one plot to keep it simple.
- Widgets will be placed on the dashboard in the order they're added. Users can specify grid positions or adjust them later in the UI.
- Iterating through multiple items to create multiple widgets is possible; however, for demonstration purposes, we make one of each widget type.
- You can adapt this notebook to use different models, making it versatile for various use cases.


For more information on specific mutations and input fields, check out our [GraphQL Documentation](https://app.arize.com/graphql).

### Step 1: Initialize the GraphQL Client using your developer API key

In [ ]:
!pip install -q gql[all]
from gql import Client, gql
from gql.transport.requests import RequestsHTTPTransport
import pandas as pd
import time

### Get your API key
First, make sure you have developer permissions. If you are able to visit the [API explorer](https://app.arize.com/graphql), then you have developer permissions. If not, please ask your Account Admin to provide you with access.

The API key can be retrieved from the [API explorer](https://app.arize.com/graphql) page. Click the button on the top right called "Get Your API Key." A modal will pop up with your key, copy that into the `API_KEY` constant below.

NOTE: this key is different than the SDK key used to send data to Arize.

In [ ]:
API_KEY = ""

# Select your transport with a defined URL endpoint
transport = RequestsHTTPTransport(
    url="https://app.arize.com/graphql/", headers={"x-api-key": API_KEY}
)

# Create a GraphQL client using the defined transport
client = Client(transport=transport, fetch_schema_from_transport=True)

## Get Model Metadata

Retrieve the metadata for the model you wish to make a dashboard for. This information will be used in the dashboard creation.

In [ ]:
MODEL_ID = ""
MODEL_ENV = "production"

In [ ]:
model_query = gql(
    """
  query getModelMetaData($modelID:ID!){
  node(id: $modelID) {
    ... on Model {
      name
      modelType
      modelEnvironments{
        environmentId
        name
      }
      uri
    }
  }
}
    """
)


# Base query parameters for fetching monitors
params = {"modelID": str(MODEL_ID)}

# An array of monitors that we will append to
metadata = []
model_name = ""


paged_response = client.execute(model_query, params)

In [ ]:
df = pd.json_normalize(paged_response, record_path=None)

df = df.explode("node.modelEnvironments").reset_index(drop=True)

model_env_df = pd.json_normalize(df["node.modelEnvironments"])

result_df = pd.concat(
    [df.drop(columns=["node.modelEnvironments"]), model_env_df], axis=1
)

In [ ]:
environmentId_value = result_df.loc[
    result_df["name"] == MODEL_ENV, "environmentId"
].item()
environmentId_value

###  Feature Retrieval

Retrieving features is crucial for creating widgets that reflect specific model dimensions.

In [ ]:
features_query = gql(
    """
    query getFeatures($modelId: ID!, $cursor: String) {
        model: node(id: $modelId) {
            ... on Model {
                name
              	id
              	modelEnvironments{
                  environmentId
                  name
                }
                modelSchema {
                    features(first: 20, after: $cursor, filter: { exclude:{ dataTypes:EMBEDDING }}) {
                        edges {
                            feature: node {
                                dimension {
                                  id
                                    name,
                                    dataType,
                                  category
                                }
                            }
                        }
                        pageInfo {
                            endCursor
                        }
                    }
                }
            }
        }
    }

"""
)

# Base query parameters for fetching features
params = {"modelId": MODEL_ID, "cursor": None}
# An array of features that we will append to
feature_data = []
model_name = ""

# Execute the query on the transport. Continue to pull data until there is no more features
while True:
    paged_response = client.execute(features_query, params)
    model_name = paged_response["model"]["name"]
    # Append the monitors to your list
    feature_data.extend(
        paged_response["model"]["modelSchema"]["features"]["edges"]
    )
    # If there is another page of information, point the cursor to the next page and fetch more
    end_cursor = paged_response["model"]["modelSchema"]["features"]["pageInfo"][
        "endCursor"
    ]
    print("pageInfo end_cursor %s" % (end_cursor))
    if end_cursor:
        print("There is another page of features. Loading more.")
        params["cursor"] = end_cursor
    else:
        print("No more features to pull. The list is complete!")
        break
    time.sleep(1)

print("Retrieved {} features".format(len(feature_data)))

In [ ]:
features = pd.json_normalize(feature_data, sep=".")
features.head()

## Dashboard Creation

Creating a new custom dashboard involves several steps, starting with initializing an empty dashboard state. Below we'll walk through each type of widget you can add to your dashboard.


### Initialize Dashboard

When creating a new custom dashboard, the first step is to create an empty state dashboard using the `createDashboard` mutation. This mutation provides you with a `dashboardId` which will be used subsequently when adding widgets.


In [ ]:
create_mutation = gql("""
mutation createDashTest($name: String!, $spaceId: ID!) {
  createDashboard(input: { name: $name, spaceId: $spaceId }) {
    dashboard {
      id
    }
  }
}
""")

In [ ]:
new_dashboard_name = "Test Dashboard"
SPACE_ID = ""
params = {"name": new_dashboard_name, "spaceId": SPACE_ID}
dashboard_response = client.execute(create_mutation, params)

In [ ]:
DASHBOARD_ID = dashboard_response["createDashboard"]["dashboard"]["id"]

### Distribution Widgets

To visualize data variations and overall distribution patterns, you can use the `createBarChartWidget` mutation. This widget is ideal for displaying the distribution of a specific dimension or metric across your dataset.

In [ ]:
widget_title = "Distribution of State"
dimension = "state"
dimension_id = features.loc[features["feature.dimension.name"] == dimension][
    "feature.dimension.id"
].item()
dimension_datatype = features.loc[
    features["feature.dimension.name"] == dimension
]["feature.dimension.dataType"].item()
dimension_category = features.loc[
    features["feature.dimension.name"] == dimension
]["feature.dimension.category"].item()

In [ ]:
params = {
    "title": widget_title,
    "dashboardId": DASHBOARD_ID,
    "plots": [
        {
            "title": dimension,
            "position": 1,
            "modelId": MODEL_ID,
            "dimension": {
                "id": dimension_id,
                "name": dimension,
                "dataType": dimension_datatype,
            },
            "dimensionCategory": dimension_category,
            "filters": [],
            "modelVersionIds": [],
        }
    ],
}

In [ ]:
createBarChartWidgetMutation = gql("""

mutation createBarChartWidget(
$title:String!
  $dashboardId:ID!
  $plots:[BarChartPlotInputInput!]!


){
  createBarChartWidget(
    input:
  {
    title:$title
    dashboardId: $dashboardId # returned from createDashboard
    plots: $plots
  }
  ){
    barChartWidget{
      id

    }
  }
}
""")

In [ ]:
result = client.execute(createBarChartWidgetMutation, params)

### Time Series Widget - Peromance Metrics
To add a line chart that tracks performance metrics like AUC or accuracy, use the createLineChartWidget mutation. Specify each metric you wish to track, and position them accordingly on your dashboard.

In [ ]:
widget_title = "AUC"
metric = "auc"
positive_class = "fraud"
plot_title = "auc"

In [ ]:
params = {
    "title": widget_title,
    "dashboardId": DASHBOARD_ID,
    "timeSeriesMetricType": "evaluationMetric",
    "plots": {
        "modelId": MODEL_ID,
        "modelVersionIds": [],
        "dimensionCategory": "prediction",
        "metric": metric,
        "positiveClass": positive_class,
        "title": plot_title,
        "position": 1,
        "modelEnvironmentName": MODEL_ENV,
        "filters": [],
    },
}

In [ ]:
createLineChartWidgetMutation = gql("""
mutation createLineChartWidget(
  $title:String!
  $dashboardId:ID!
  $plots:[LineChartPlotInputInput!]!
  $timeSeriesMetricType: TimeSeriesMetricCategory!
){
  createLineChartWidget(
    input:
  {
    title:$title
    timeSeriesMetricType: $timeSeriesMetricType
    dashboardId: $dashboardId
    plots: $plots
    widgetType: lineChartWidget
  }
  ){
    lineChartWidget{
      id
    }
  }
}
""")

In [ ]:
result = client.execute(createLineChartWidgetMutation, params)

### Time Series Widget - Data Quality Metrics

Similarly, for data quality metrics such as missing data percentages or outlier counts, utilize the same mutation. Configuration may include specifying dimensions and metrics that are relevant to your data quality analysis.

In [ ]:
widget_title = "Percent Empty"
metric = "percentEmpty"
plot_title = "percent Empty for merchant_id"
dimension = "merchant_ID"
dimension_id = features.loc[features["feature.dimension.name"] == dimension][
    "feature.dimension.id"
].item()
dimension_datatype = features.loc[
    features["feature.dimension.name"] == dimension
]["feature.dimension.dataType"].item()
dimension_category = features.loc[
    features["feature.dimension.name"] == dimension
]["feature.dimension.category"].item()

In [ ]:
params = {
    "title": widget_title,
    "dashboardId": DASHBOARD_ID,
    "timeSeriesMetricType": "evaluationMetric",
    "plots": {
        "modelId": MODEL_ID,
        "modelVersionIds": [],
        "dimensionCategory": dimension_category,
        "metric": metric,
        "dimension": {
            "id": dimension_id,
            "name": dimension,
            "dataType": dimension_datatype,
        },
        "title": plot_title,
        "position": 1,
        "modelEnvironmentName": MODEL_ENV,
        "filters": [],
    },
}

In [ ]:
createDQLineChartWidgetMutation = gql("""
mutation createLineChartWidget(
  $title:String!
  $dashboardId:ID!
  $plots:[LineChartPlotInputInput!]!
  $timeSeriesMetricType: TimeSeriesMetricCategory!
){
  createLineChartWidget(
    input:
  {
    title:$title
    timeSeriesMetricType: $timeSeriesMetricType
    dashboardId: $dashboardId
    plots: $plots
    widgetType: lineChartWidget
  }
  ){
    lineChartWidget{
      id
    }
  }
}
""")

In [ ]:
result = client.execute(createDQLineChartWidgetMutation, params)

### Drift Widget
Monitoring data drift is crucial for maintaining model performance. Use the `createLineChartWidget` mutation with a `widgetType` set to `driftLineChartWidget` to track drift using various statistical methods like PSI, JS divergence, or KL divergence.

First, retrieve a baseline ID, which is essential for comparing current model performance against a historical benchmark:

In [ ]:
getModelBaselineMutation = gql(
    """
query getBaseline($modelID:ID!){
  node(id: $modelID) {
    ... on Model {
      modelPrimaryBaseline{
        id
      }
    }
  }
}
    """
)

baseline_response = client.execute(
    getModelBaselineMutation, {"modelID": MODEL_ID}
)
BASELINE_ID = baseline_response["node"]["modelPrimaryBaseline"]["id"]

In [ ]:
widget_title = "Prediction Drift"
metric = "psi"
plot_title = ""

In [ ]:
params = {
    "title": widget_title,
    "dashboardId": DASHBOARD_ID,
    "timeSeriesMetricType": "evaluationMetric",
    "plots": [
        {
            "modelId": MODEL_ID,
            "modelVersionIds": [],
            "dimensionCategory": "prediction",
            "metric": metric,
            "title": plot_title,
            "position": 1,
            "modelEnvironmentName": MODEL_ENV,
            "filters": [],
            "comparisonDatasetModelBaselineId": BASELINE_ID,
        }
    ],
}

In [ ]:
createDriftWidgetMutation = gql(
    """
    mutation createDriftWidget(
    $title:String!
      $dashboardId:ID!
      $plots:[LineChartPlotInputInput!]!
      $timeSeriesMetricType: TimeSeriesMetricCategory!
    ){
      createLineChartWidget(
        input:
      {
        title:$title
        timeSeriesMetricType: $timeSeriesMetricType
        dashboardId: $dashboardId
        plots: $plots
        widgetType: driftLineChartWidget
      }
      ){
        lineChartWidget{
          id
        }
      }
    }
    """
)

In [ ]:
client.execute(createDriftWidgetMutation, params)

### Statistic Widget

Add a statistics widget to provide a quick overview of key metrics. This widget can focus on either performance indicators or data quality metrics, offering valuable insights at a glance. Configure this widget to display counts, averages, or other statistical summaries of specified dimensions.

In [ ]:
widget_title = "count of fico score"
metric = "count"
dimension = "fico_score"
dimension_id = features.loc[features["feature.dimension.name"] == dimension][
    "feature.dimension.id"
].item()
dimension_datatype = features.loc[
    features["feature.dimension.name"] == dimension
]["feature.dimension.dataType"].item()
dimension_category = features.loc[
    features["feature.dimension.name"] == dimension
]["feature.dimension.category"].item()

In [ ]:
params = {
    "title": widget_title,
    "dashboardId": DASHBOARD_ID,
    "timeSeriesMetricType": "modelDataMetric",
    "modelId": MODEL_ID,
    "aggregation": metric,
    "dimension": {
        "id": dimension_id,
        "name": dimension,
        "dataType": dimension_datatype,
    },
    "dimensionCategory": dimension_category,
    "filters": [],
}

In [ ]:
createStatWidgetMutation = gql(
    """
mutation createStatWidget(
  $title:String!
  $dashboardId:ID!
  $timeSeriesMetricType: TimeSeriesMetricCategory!
  $modelId:ID!
  $aggregation: DataQualityMetric
  $dimension:DimensionInput
  $dimensionCategory: DimensionCategory
  $filters: [StatisticWidgetFilterItemInput!]!

){
  createStatisticWidget(
    input:
  {
    dashboardId: $dashboardId
    title:$title
    timeSeriesMetricType:$timeSeriesMetricType
    modelId:$modelId
    aggregation:$aggregation
    dimensionCategory:$dimensionCategory
    dimension:$dimension
    filters:$filters
    creationStatus:published
    modelVersionEnvironmentMetadataIds:[]
    modelVersionIds:[]
  })
  {
    statisticWidget{
      id
    }
  }
}
"""
)

In [ ]:
client.execute(createStatWidgetMutation, params)

### Text Widget

Add a text widget to your dashboard to offer explanatory text, headings, or contextual information, enriching the dashboard's narrative and usability.

In [ ]:
params = {
    "title": "text-widget-title",
    "dashboardId": DASHBOARD_ID,
    "content": "content for the text widget",
}

In [ ]:
createTextWidgetMutation = gql(
    """
mutation createTextWidget(
  $title:String!
  $dashboardId:ID!
  $content:String!

){
  createTextWidget(
    input:
  {
    dashboardId: $dashboardId
    title:$title
    content: $content
    creationStatus:published

  })
  {
    textWidget{
      id
    }
  }
}
"""
)

In [ ]:
client.execute(createTextWidgetMutation, params)